In [ ]:
import requests
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [ ]:
product_name=input('Enter the product: ')

Enter the product: smartphones


In [ ]:
base_url='https://www.amazon.in'
url=f'https://www.amazon.in/s?k={product_name}&crid=32B7BW3NO3GPO&sprefix=laptops%2Caps%2C234&ref=nb_sb_noss_1'
headers={
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

In [ ]:
base_response=requests.get(base_url,headers=headers)
cookies=base_response.cookies
product_response=requests.get(url,headers=headers,cookies=cookies)

In [ ]:

def amazon_data(product_response):
    soup = BeautifulSoup(product_response.content, 'html.parser')

    price_lines = [tag.get_text() for tag in soup.findAll("span", {"class": 'a-price-whole'})]
    names = [tag.get_text() for tag in soup.findAll("span", {"class": 'a-size-medium a-color-base a-text-normal'})]
    div_elements = soup.find_all("div", {"class": "a-row a-size-small"})
    ratings = []
    for div in div_elements:
        rating_element = div.find("span", {"class": "a-icon-alt"})
        if rating_element:
            ratings.append(rating_element.get_text().replace('out of 5 stars',''))
        else:
            ratings.append(None)
    span_elements = soup.find_all("span", {"class": "a-price a-text-price"})
    original_price = [span.find("span", {"class": 'a-offscreen'}).get_text() if span.find("span", {"class": 'a-offscreen'}) else None for span in span_elements]

    min_length = min(len(names), len(price_lines), len(ratings), len(original_price))
    names = names[:min_length]
    price_lines = price_lines[:min_length]
    ratings = ratings[:min_length]
    original_price = original_price[:min_length]

    return names, price_lines, ratings, original_price

names, price_lines, ratings, original_price = amazon_data(product_response)

data = {
    'Name': names,
    'Price': price_lines,
    'Original Price': original_price,
    'Rating': ratings
}

df1 = pd.DataFrame(data)


In [ ]:
df1.head(10)

,Name,Price,Original Price,Rating
0,"Samsung Galaxy M13 (Aqua Green, 4GB, 64GB Stor...","9,999","₹14,999",4.1
1,"Oneplus Nord CE 3 5G (Grey Shimmer, 8GB RAM, 1...","22,999","₹26,999",4.2
2,"Samsung Galaxy M34 5G (Waterfall Blue,8GB,128G...","16,999","₹25,999",4.0
3,"AMBLIC Gimbal for Phones with LED Fill Lighgt,...","3,099","₹4,999",4.0
4,"realme narzo N53 (Feather Black, 4GB+64GB) 33W...","7,499","₹10,999",4.3
5,"OnePlus 12R (Iron Gray, 8GB RAM, 128GB Storage)","39,999","₹13,999",4.1
6,"Redmi 13C (Stardust Black, 6GB RAM, 128GB Stor...","8,499","₹13,999",4.0
7,"realme narzo N53 (Feather Gold, 8GB+128GB) 33W...","8,999","₹31,999",4.0
8,"realme 12 Pro 5G (Submarine Blue, 8GB RAM 256 ...","23,589","₹20,999",3.6
9,"Redmi Note 13 5G (Stealth Black, 6GB RAM, 128G...","17,999","₹22,000",3.9


# WEB SCRAPING FROM SNAP DEAL


# Good Code - Snapdeal


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrape_snapdeal(product_name):
    base_url = "https://www.snapdeal.com/search?keyword={}&santizedKeyword={}&catId=0&categoryId=0&suggested=false&vertical=p&noOfResults=20&searchState=&clickSrc=go_header&lastKeyword=&prodCatId=&changeBackToAll=false&foundInAll=false&categoryIdSearched=&cityPageUrl=&categoryUrl=&url=&utmContent=&dealDetail=&sort=rlvncy"
    formatted_product_name = '+'.join(product_name.split())
    url = base_url.format(formatted_product_name, formatted_product_name)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    products = soup.find_all("div", class_="product-tuple-listing")

    data = []

    for product in products:
        title = product.find("p", class_="product-title").text.strip()
        price = product.find("span", class_="product-price").text.strip()

        original_price_tag = product.find("span", class_="product-desc-price strike")
        if original_price_tag:
            original_price = original_price_tag.text.strip()
        else:
            original_price = None

        discount_tag = product.find("div", class_="product-discount")
        if discount_tag:
            discount = discount_tag.text.strip()
        else:
            discount = None

        rating_tag = product.find("div", class_="filled-stars")
        if rating_tag:
            rating = rating_tag['style'].split(":")[-1].split("%")[0]
        else:
            rating = None

        rating_count_tag = product.find("p", class_="product-rating-count")
        if rating_count_tag:
            rating_count = rating_count_tag.text.strip()[1:-1]
        else:
            rating_count = None

        data.append({
            "Title": title,
            "Price_": price,
            "Original Price_": original_price,
            "Discount_": discount,
            "Rating_": rating + "%" if rating else "Not Available",
            "Rating Count": rating_count if rating_count else "Not Available"
        })

    df = pd.DataFrame(data)
    return df


df2 = scrape_snapdeal(product_name)



In [ ]:
df2.head(10)

,Title,Price_,Original Price_,Discount_,Rating_,Rating Count
0,SLOVIC® Mobile Mount Adapter | Mobile Holder |...,Rs. 209,None,74% Off,86.0%,131
1,Tecsox Bluetooth Selfie Stick ( Black ),Rs. 649,None,68% Off,82.0%,69
2,"NBOX Wall Mount Mobile Holder, Mobile Stand fo...",Rs. 199,None,67% Off,92.0%,56
3,Tecsox - Adjustable Mobile Stand for Smartphon...,Rs. 249,None,75% Off,90.0%,203
4,NBOX Selfie Ring Light 36 LED Clip-on Smartpho...,Rs. 249,None,75% Off,80.0%,607
5,Tecsox - Foldable Mobile Stand for Smartphones...,Rs. 639,None,68% Off,88.00000000000001%,33
6,NBOX - Foldable Mobile Holder for Smartphones ...,Rs. 149,None,70% Off,82.0%,432
7,Portronics POR-122 MODESK Universal Mobile H...,Rs. 159,None,77% Off,86.0%,244
8,Elv Foldable Mobile Holder for Smartphones and...,Rs. 279,None,81% Off,80.0%,1
9,NBOX Adjustable Mobile Holder for Smartphones ...,Rs. 179,None,64% Off,60.0%,2


In [ ]:
df2.shape

(20, 6)

In [ ]:
def extract_numeric(discount_str):
    numeric_value = re.search(r'\d+', discount_str).group()
    return int(numeric_value)

df2['Discount'] = df2['Discount_'].apply(extract_numeric)


In [ ]:
def extract_numeric(price_str):
    numeric_value = re.sub(r'[^\d]', '', price_str)
    return int(numeric_value)

df2['Price'] = df2['Price_'].apply(extract_numeric)

In [ ]:
df2['Price']

0     209
1     649
2     199
3     249
4     249
5     639
6     149
7     159
8     279
9     179
10    199
11    379
12    110
13    379
14    159
15    299
16    349
17    299
18    589
19    199
Name: Price, dtype: int64

In [ ]:
def calculate_original_price(row):
    discount_percent = row['Discount'] / 100
    original_price = row['Price'] / (1 - discount_percent)
    return original_price

df2['Price'] = df2.apply(calculate_original_price, axis=1)



In [ ]:
df2['Price']

0      803.846154
1     2028.125000
2      603.030303
3      996.000000
4      996.000000
5     1996.875000
6      496.666667
7      691.304348
8     1468.421053
9      497.222222
10    1326.666667
11     997.368421
12     500.000000
13    1516.000000
14     496.875000
15    1495.000000
16    1517.391304
17     996.666667
18    2031.034483
19     603.030303
Name: Price, dtype: float64

In [ ]:
df2['Price'] = df2['Price'].round().astype(int)


In [ ]:
def convert_to_stars(rating_str):
    if rating_str == 'Not Available':
        return 'No ratings'
    else:
        rating_float = float(rating_str.strip('%')) / 20
        return round(rating_float, 1)

df2['Stars'] = df2['Rating_'].apply(convert_to_stars)

In [ ]:
df2=df2.drop(['Price_','Original Price_','Discount_','Rating_'],axis=1)

In [ ]:
df2

,Title,Rating Count,Discount,Price,Stars
0,SLOVIC® Mobile Mount Adapter | Mobile Holder |...,131,74,804,4.3
1,Tecsox Bluetooth Selfie Stick ( Black ),69,68,2028,4.1
2,"NBOX Wall Mount Mobile Holder, Mobile Stand fo...",56,67,603,4.6
3,Tecsox - Adjustable Mobile Stand for Smartphon...,203,75,996,4.5
4,NBOX Selfie Ring Light 36 LED Clip-on Smartpho...,607,75,996,4.0
5,Tecsox - Foldable Mobile Stand for Smartphones...,33,68,1997,4.4
6,NBOX - Foldable Mobile Holder for Smartphones ...,432,70,497,4.1
7,Portronics POR-122 MODESK Universal Mobile H...,244,77,691,4.3
8,Elv Foldable Mobile Holder for Smartphones and...,1,81,1468,4.0
9,NBOX Adjustable Mobile Holder for Smartphones ...,2,64,497,3.0


# Shop Clues


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://www.shopclues.com/'

url = f'https://www.shopclues.com/search?q={product_name}'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

base_response = requests.get(base_url, headers=headers)
cookies = base_response.cookies
product_response = requests.get(url, headers=headers, cookies=cookies)


def shopclue_data(product_response):
    soup = BeautifulSoup(product_response.content, 'html.parser')

    names = [tag.get_text(strip=True) for tag in soup.findAll("h2", {"class": ''})]

    price_elements = soup.findAll("span", {"class": "p_price"})
    price_lines = [price_element.get_text(strip=True) for price_element in price_elements]

    original_price_elements = soup.findAll("span", {"class": "old_prices"})
    original_prices = [original_price_element.get_text(strip=True) if original_price_element.find("span") else None for original_price_element in original_price_elements]

    discount_elements = soup.findAll("span", {"class": "prd_discount"})
    discounts = [discount_element.get_text(strip=True) if discount_element else None for discount_element in discount_elements]

    star_elements = soup.findAll("div", {"class": "rtnrew"})
    stars = []
    for star_element in star_elements:
        star_span = star_element.find("span", {"style": True})
        if star_span:
            width = float(star_span['style'].split(":")[1].replace('px', '').strip())
            stars.append(width / 20)
        else:
            stars.append(None)

    return names, price_lines, original_prices, discounts, stars


names, price_lines, original_prices, discounts, stars = shopclue_data(product_response)
min_length = min(len(names), len(price_lines), len(original_prices), len(discounts), len(stars))
names = names[:min_length]
price_lines = price_lines[:min_length]
original_prices = original_prices[:min_length]
discounts = discounts[:min_length]
stars = stars[:min_length]

data = {'Product Name': names, 'Price': price_lines, 'Original Price': original_prices, 'Discount': discounts, 'Stars': stars}
df3 = pd.DataFrame(data)




In [ ]:
df3.head(10)

,Product Name,Price,Original Price,Discount,Stars
0,"Nokia C20 Plus Smartphone (Ocean Blue, 32 GB)(...","₹8,810","₹11,099",21% Off,NaN
1,"Nokia C20 Plus Smartphone (Dark Grey, 32 GB)(3...","₹8,810","₹11,099",21% Off,3.01
2,Nokia C20 Plus Smartphone Dark Grey 32 Gb2 Gb Ram,"₹7,999","₹10,099",21% Off,2.10
3,"Nokia C20 Plus Smartphone (Ocean Blue, 32 GB) ...","₹8,499","₹11,999",29% Off,NaN
4,Global ShoppingSee More,"₹4,799","₹9,999",52% Off,1.89
5,Nokia 2660 4G Flip Smartphone,"₹8,499","₹11,999",29% Off,NaN
6,"Nokia C20 Plus Smartphone (Dark Grey, 32 GB) ...","₹7,499","₹8,899",16% Off,NaN
7,"Itel P40 (4 GB RAM, 64 GB Storage, Dreamy Blue)","₹8,999","₹10,999",18% Off,NaN
8,"Nokia 2.3 (Charcoal, 32 GB) (2 GB RAM)","₹10,490","₹16,990",38% Off,2.03
9,"OPPO A57 (6 GB RAM, 128 GB Storage)","₹12,269","₹13,999",12% Off,NaN
